chmod 600 ~/.ssh/id_ed25519### Current Process
1. Read in data --> Done

2. Custom Imputation --> Done

3. Add Binary Class --> Done, Should Add Binary Class Later

4. Summary Statistics Features --> Done

5. Wrapper Functions --> Done, Need to Test Though

6. Sklearn Pipeline Categorical Features --> One Hot Encoding Done

7. Sklearn Pipeline Numerical Features --> StandardScaler Done

8. Create Lagged Features --> Done

9. Modeling --> Currently XgBoost, (Maybe Try: TensorFlow Decision Tree, TensorFlow Probability Model)

10. Model Evaluation --> Accuracy, Precision, Recall, F1, Confusion Matrix (Need to add Variable Importance Based on Variance)

11. PySpark: XGBoost Classification Feature Importance

In [ ]:
# # Need to Run These in Notebook Version For Pandas UDF
! pip install pyarrow
! pip install pandas
! pip install scikit-learn
! pip install pyspark
! pip install xgboost
! pip install kaleido
! pip install EntropyHub

In [1]:
from Input_Variables.read_vars import train_data_storage, validation_data_storage, test_data_storage, \
                                      inter_train_location, inter_test_location, inter_val_location,\
                                      one_hot_encoding_data, \
                                      analysis_group, \
                                      daily_stats_features_lower, daily_stats_features_upper, \
                                      model_storage_location, random_seed, \
                                      time_series_lag_values_created, \
                                      evaluation_metrics_output_storage, \
                                      feature_importance_storage_location, \
                                      overall_feature_importance_plot_location

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.imputation_pipeline import Date_And_Value_Imputation


from Feature_Generation.create_binary_labels import Create_Binary_Labels
from Feature_Generation.summary_stats import Summary_Stats_Features
from Feature_Generation.lag_features import Create_Lagged_Features
from Feature_Generation.time_series_feature_creation import TS_Features
from Feature_Generation.difference_features import Difference_Features

from Data_Pipeline.encoding_scaling_pipeline import Feature_Transformations

from Model_Creation.pyspark_xgboost import Create_PySpark_XGBoost

from Model_Predictions.pyspark_model_preds import Model_Predictions

from Model_Evaluation.pyspark_model_eval import Evaluate_Model

from Feature_Importance.model_feature_importance import Feature_Importance

from Model_Plots.xgboost_classification_plots import XGBoost_Classification_Plot

import os

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data()

# Create Binary y Variables
create_binary_labels=Create_Binary_Labels()

# Imputation
date_and_value_imputation=Date_And_Value_Imputation()

# Features Daily Stats Module
summary_stats_features=Summary_Stats_Features()

# Features Complex
ts_features=TS_Features()

# Features Lagged Value
create_lag_features=Create_Lagged_Features()

# Features Differences
difference_features=Difference_Features()

# PySpark XGBoost Model Module
create_pyspark_xgboost=Create_PySpark_XGBoost()

# Classification Evaluation
evaluate_model=Evaluate_Model()

# Model Plots Feature Importance
xgboost_classification_plot=XGBoost_Classification_Plot()

# Feature Transformations
feature_transformations=Feature_Transformations()


pyspark_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()


model_predictions=Model_Predictions()

# Feature Importance
feature_importance=Feature_Importance()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/16 09:30:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# PySpark

### 1. PySpark: Reading In Data

#### Training

In [ ]:
training_df=reading_data.read_in_pyspark_data(data_location=train_data_storage)
training_df.show()

#### Validation

In [ ]:
validation_df=reading_data.read_in_pyspark_data(data_location=validation_data_storage)
validation_df.show()

#### Testing

In [ ]:
testing_df=reading_data.read_in_pyspark_data(data_location=test_data_storage)
testing_df.show()

### 2. PySpark: Custom Imputation Pipeline

#### Training

In [3]:
interpolation_complete = os.path.exists('/cephfs/interpolation/train')

if interpolation_complete == False:
    date_and_value_imputation.interpolation_creation('train')
    
training_custom_imputation_pipeline = date_and_value_imputation.read_interpolation('/cephfs/interpolation/train/')

training_custom_imputation_pipeline.show(2)

+-------------------+-----+-----+----------+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|
+-------------------+-----+-----+----------+
|2022-02-01 00:02:00|    0|155.0|       0.0|
|2022-02-01 00:07:00|    0|155.0|       0.0|
+-------------------+-----+-----+----------+
only showing top 2 rows



#### Testing

In [ ]:
interpolation_complete = os.path.exists('/cephfs/interpolation/test')

if interpolation_complete == False:
    date_and_value_imputation.interpolation_creation('test')
    
testing_custom_imputation_pipeline = date_and_value_imputation.read_interpolation('/cephfs/interpolation/test/')

testing_custom_imputation_pipeline.show(2)

### 3. PySpark: Adding Binary Labels

#### Training

In [4]:
training_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=training_custom_imputation_pipeline)

training_df_added_binary_labels.show(1)

/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------------------+-----+-----+----------+--------+--------+--------+
| GlucoseDisplayTime|NumId|Value|IsFilledIn|y_Binary|is_above|is_below|
+-------------------+-----+-----+----------+--------+--------+--------+
|2022-02-01 00:00:00|   26|126.0|       0.0|       0|       0|       0|
+-------------------+-----+-----+----------+--------+--------+--------+
only showing top 1 row



#### Testing

In [ ]:
testing_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=testing_custom_imputation_pipeline)

testing_df_added_binary_labels.show(1, truncate=False)

### 4. PySpark: Feature Creation

#### Training

##### Complex Features

In [5]:
training_df_differences = difference_features.add_difference_features(training_custom_imputation_pipeline)
training_df_differences.show(5)

+-------------------+-----+---------+----------+---------+----------+
| GlucoseDisplayTime|NumId|    Value|IsFilledIn|FirstDiff|   SecDiff|
+-------------------+-----+---------+----------+---------+----------+
|2022-02-01 00:00:00|   26|    126.0|       0.0|      0.0|       0.0|
|2022-02-01 00:05:00|   26|    123.0|       0.0|     -3.0|      -3.0|
|2022-02-01 00:10:00|   26|    123.0|       0.0|      0.0|       3.0|
|2022-02-01 00:15:00|   26|136.60336|       1.0|13.603363| 13.603363|
|2022-02-01 00:20:00|   26|136.60336|       1.0|      0.0|-13.603363|
+-------------------+-----+---------+----------+---------+----------+
only showing top 5 rows



In [6]:
training_df_chunks = summary_stats_features.create_chunk_col(training_df_differences, chunk_val = 288)
training_df_chunks.show(5)

+-------------------+-----+---------+----------+---------+----------+-----+-----+
| GlucoseDisplayTime|NumId|    Value|IsFilledIn|FirstDiff|   SecDiff|index|Chunk|
+-------------------+-----+---------+----------+---------+----------+-----+-----+
|2022-02-01 00:00:00|   26|    126.0|       0.0|      0.0|       0.0|    1|    0|
|2022-02-01 00:05:00|   26|    123.0|       0.0|     -3.0|      -3.0|    2|    0|
|2022-02-01 00:10:00|   26|    123.0|       0.0|      0.0|       3.0|    3|    0|
|2022-02-01 00:15:00|   26|136.60336|       1.0|13.603363| 13.603363|    4|    0|
|2022-02-01 00:20:00|   26|136.60336|       1.0|      0.0|-13.603363|    5|    0|
+-------------------+-----+---------+----------+---------+----------+-----+-----+
only showing top 5 rows



In [7]:
training_df_poincare = training_df_chunks.groupby(['NumId', 'Chunk']).apply(ts_features.poincare)
training_df_poincare.show(5)

training_df_entropy = training_df_chunks.groupby(['NumId', 'Chunk']).apply(ts_features.entropy)
training_df_entropy.show(5)

+-----+-----+-----------------+----------------+-------------+
|NumId|Chunk|ShortTermVariance|LongTermVariance|VarianceRatio|
+-----+-----+-----------------+----------------+-------------+
|   26|    0|        21.724335|       12.153462|    1.7875017|
|   26|    1|         51.52916|       30.086325|    1.7127104|
|   26|    2|         38.08224|        21.94321|    1.7354909|
|   26|    3|         32.44667|       21.120747|    1.5362464|
|   26|    4|         32.88985|       16.970528|    1.9380571|
+-----+-----+-----------------+----------------+-------------+
only showing top 5 rows



23/05/16 10:06:19 WARN ArrowPythonRunner: Detected deadlock while completing task 0.0 in stage 36 (TID 2226): Attempting to kill Python Worker
+-----+-----+----------+
|NumId|Chunk|   Entropy|
+-----+-----+----------+
|   26|    0|0.52779317|
|   26|    1| 0.6031273|
|   26|    2|0.55217344|
|   26|    3|0.28888184|
|   26|    4|0.33758768|
+-----+-----+----------+
only showing top 5 rows



In [ ]:
# karina = training_df_chunks.where(training_df_chunks.NumId=='26')
# karina = karina.where(karina.Chunk=='0')
# karina.show(2)

In [ ]:
# patientid = karina['NumId'].iloc[0]
# chunk = karina['Chunk'].iloc[0]
# kk = karina.groupBy(['NumId', 'Chunk'])
# kk.printSchema

# glucose_differentials = np.diff(kk.Value)
# st_dev_differentials = np.std(np.diff(glucose_differentials))
# st_dev_values = np.std(glucose_differentials)

# # measures the width of poincare cloud
# short_term_variation = round((1 / np.sqrt(2)) * st_dev_differentials, 3)

# # measures the length of the poincare cloud
# long_term_variation = round(np.sqrt((2 * st_dev_values ** 2) - (0.5 * st_dev_differentials ** 2)), 3)

# ratio = round(short_term_variation / long_term_variation, 3)

# poincare_df = pd.DataFrame([[patientid] + [chunk] + [short_term_variation] + [long_term_variation] + [ratio]])
# poincare_df.columns=['NumId', 'Chunk', 'ShortTermVariance', 'LongTermVariance', 'VarianceRatio']
# poincare_df.show(3)

In [ ]:
# training_df_complex_features = training_df_poincare.join(training_df_entropy,['PatientId', 'Chunk'])
# training_df_complex_features.show()

In [ ]:
# training_df_sleep = ts_features.process_for_sleep(df=training_df_added_binary_labels)
# training_df_sleep.show(5)

##### Statistical Features

In [ ]:
training_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=training_df_chunks)
# merge complex features and summary stats and demographics and sleep features
# merge in one hot encoded cohort file info demographics
    # '/cephfs/data/cohort_encoded.parquet' (gender, treatment, age category)
    # groupby patientId and chunk

training_features_summary_stats.show(3)

In [ ]:
# #add target variable
# training_features_final_summary = summary_stats_features\
#                                     .add_lag_out_of_range(df=training_features_summary_stats, chunk_lag=1)

# training_features_final_summary.show(4)

In [ ]:
# Merge these together
# training_features_summary_stats
# training_df_complex_features
# one-hot-encoding 

#### Testing

##### Complex Features

In [ ]:
testing_df_differences = difference_features.add_difference_features(testing_df_added_binary_labels)
testing_df_differences.show(5)

In [ ]:
testing_df_chunks = summary_stats_features.create_chunk_col(testing_df_differences, chunk_val = 288)
testing_df_chunks.show(5)

# testing_df_poincare = testing_df_chunks.groupby(['PatientId', 'Chunk']).apply(ts_features.poincare)
# testing_df_poincare.show(5)

# testing_df_entropy = testing_df_chunks.groupby(['PatientId', 'Chunk']).apply(ts_features.entropy)
# testing_df_entropy.show(5)

In [ ]:
# testing_df_complex_features = testing_df_poincare.join(testing_df_entropy,['PatientId', 'Chunk'])
# testing_df_complex_features.show()

In [ ]:
# training_df_sleep = ts_features.process_for_sleep(df=testing_df_added_binary_labels)
# training_df_sleep.show(5)

##### Statistical Features

In [ ]:
testing_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=testing_df_chunks)

# merge complex features and summary stats and demographics and sleep features
# merge in one hot encoded cohort file info demographics
    # '/cephfs/data/cohort_encoded.parquet' (gender, treatment, age category)
    # groupby patientId and chunk

testing_features_summary_stats.show(3)

In [ ]:
# Merge these together
# testing_features_summary_stats
# training_df_complex_features
# one-hot-encoding 

### 7. PySpark: Sklearn Regression Pipeline in PySpark

In [ ]:
one_hot_encoded_df=reading_data.read_in_one_hot_encoded_data(one_hot_encoding_location=one_hot_encoding_data)
one_hot_encoded_df=one_hot_encoded_df.select('UserId', 
                                             'Sex_Encoded', 
                                             'Treatment_Encoded', 
                                             'AgeGroup_Encoded')

In [ ]:
one_hot_encoded_df.show(4)

In [ ]:
vecAssembler = VectorAssembler(outputCol="features")
vecAssembler.setInputCols(["Sex_Encoded", "Treatment_Encoded", "AgeGroup_Encoded"])

test = vecAssembler.transform(one_hot_encoded_df)

In [ ]:
test

In [ ]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(test)

#### Training

In [ ]:
training_encoded=training_features_summary_stats.join(one_hot_encoded_df,
                                                       training_features_summary_stats.PatientId==one_hot_encoded_df.UserId,
                                                       "left")


In [ ]:
training_encoded.show(4)

In [ ]:
# merge training_features_summary with 

#### Testing

In [ ]:
testing_encoded=testing_features_summary_stats.join(one_hot_encoded_df,
                                                       testing_features_summary_stats.PatientId==one_hot_encoded_df.UserId,
                                                       "left")

In [ ]:
testing_encoded.show(4)

### 8. PySpark: Sklearn Numerical Pipeline in PySpark

In [ ]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.sql.types import DoubleType, FloatType, LongType
from pyspark.ml import Pipeline

In [ ]:
training_encoded.select('PatientId').show(1, truncate=False)

In [ ]:
patient_1=training_encoded.filter(training_encoded.PatientId=='8W/rpnb48OMm47W2x4FSkc7+9u2mol061DQuJoMdiK0=')

In [ ]:
double_cols=[f.name for f in patient_1.schema.fields if isinstance(f.dataType, DoubleType)]
float_cols=[f.name for f in patient_1.schema.fields if isinstance(f.dataType, FloatType)]
long_cols=[f.name for f in patient_1.schema.fields if isinstance(f.dataType, LongType)]

# all_numerical=list(set(double_cols+float_cols))
# all_numerical_lags=[x for x in all_numerical if "lag" in x]
all_numerical=list(set(double_cols+float_cols+long_cols))
all_numerical.remove('target')

# featureArr = [('scaled_' + f) for f in all_numerical_lags]
featureArr = [('scaled_' + f) for f in all_numerical]+['Sex_Encoded', 'Treatment_Encoded', 'AgeGroup_Encoded']
# featureArr = [('scaled_' + f) for f in all_numerical]

va1 = [VectorAssembler(inputCols=[f], outputCol=('vec_' + f)) for f in all_numerical]
ss = [StandardScaler(inputCol='vec_' + f, outputCol='scaled_' + f, withMean=True, withStd=True) for f in all_numerical]

va2 = VectorAssembler(inputCols=featureArr, outputCol="features")

stages = va1 + ss + [va2]

In [ ]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(training_encoded)

In [ ]:
model.transform(training_encoded).select('features').show(3, truncate=False)

In [ ]:
model.transform(training_encoded).select('features').show(3, truncate=False)

In [ ]:
pipeline=Pipeline(stages=stages)

model=pipeline.fit(patient_1)

In [ ]:
model.transform(patient_1).select('features').show(3, truncate=False)

In [ ]:
# training_numerical_stages=feature_transformations.numerical_scaling(df=training_encoded)
training_numerical_stages=feature_transformations.numerical_scaling(df=training_features_summary_stats)




### 9. PySpark: XGBoost Model

In [ ]:
# xgboost_model=create_pyspark_xgboost.xgboost_classifier(ml_df=training_encoded,
#                                                         stages=training_numerical_stages,
#                                                         model_storage_location=model_storage_location,
#                                                         random_seed=random_seed)

xgboost_model=create_pyspark_xgboost.xgboost_classifier(ml_df=training_features_summary_stats,
                                                        stages=training_numerical_stages,
                                                        model_storage_location=model_storage_location,
                                                        random_seed=random_seed)

### 10. PySpark: Cross Validation

### 11. PySpark: Model Predictions

In [ ]:
# testing_predictions=model_predictions.create_predictions_with_model(test_df=testing_encoded, 
#                                                                     model=xgboost_model)
# testing_predictions.show(10)

testing_predictions=model_predictions.create_predictions_with_model(test_df=training_features_summary_stats, 
                                                                    model=xgboost_model)
testing_predictions.show(10)

### 12. PySpark: Model Evaluation

In [ ]:
model_evaluation=evaluate_model.regression_evaluation(testing_predictions=testing_predictions, 
                                                          eval_csv_location=evaluation_metrics_output_storage)

In [ ]:
model_evaluation.head()

### 13. PySpark: XGBoost Classification Feature Importance

In [ ]:
feature_importance_df=feature_importance.\
                        feature_importance_accuracy_gain(xgboost_model=xgboost_model, 
                                                         feature_importance_storage_location=feature_importance_storage_location)


In [ ]:
feature_importance_df.head(10)

### 14. PySpark: Feature Importance Plotting

In [ ]:
overall_feature_plot=xgboost_classification_plot.feature_overall_importance_plot(feature_importance_df=feature_importance_df,
                                                                                 overall_importance_plot_location=overall_feature_importance_plot_location)


In [ ]:
overall_feature_plot.show()

### 15.PySpark: Local Level Feature Importance --> Shap Pandas UDF

In [ ]:
# Add to reqs if this works
! pip install shap

In [ ]:
xgboost_model.stages[-1]

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(xgboost_model.stages[-1])